# Modules, colors, and functions

Remove scroll from figures

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

Extend the size of the Window

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Load Modules

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy import convolution as conv
import sunpy.cm as cm
import sunpy.map
import sunpy.physics.differential_rotation as dr
import scipy as sp
from scipy import sparse, signal, stats
from pprint import pprint
import glob
from scipy.interpolate import griddata

from skimage.filters import sobel
from skimage.morphology import watershed

%matplotlib inline

In [5]:
# Read File
HMIfile = 'Data/HMI/hmi.M_720s.20110401_000000_TAI.1.magnetogram.fits'

HMI_fits = fits.open(HMIfile, cache=False)
HMI_fits.verify('fix')

# Assemble Sunpy map
HMImap = sunpy.map.Map(HMI_fits[1].data, HMI_fits[1].header)

# Fix instrument rotation
HMImap = HMImap.rotate(rmatrix=HMImap.rotation_matrix, missing=np.nan, recenter=True)

# Apply NaNs for Pixels outside the Sun
x, y = np.meshgrid(*[np.arange(v.value) for v in HMImap.dimensions]) * u.pixel
hpc_coords = HMImap.pixel_to_world(x, y)
r = np.sqrt(hpc_coords.Tx ** 2 + hpc_coords.Ty ** 2) / HMImap.rsun_obs
HMImap.data[r>1] = np.nan

 [astropy.io.fits.verify]
